# **Homework 8 Solutions**

Due:

## **Programming Assignment Solutions**

### Introduction

In this assignment, you'll implement Naive Bayes and use this algorithm
to classify the credit rating (good or bad) of a set of individuals. The
textbook section relevant to this assignment is 24.2 on page 347.

### Stencil Code & Data

We have provided the following stencils:

-   `Models` contains the `NaiveBayes` model which you will be
    implementing.

-   `Check Model` contains a series of tests to ensure you are coding your 
    model properly.

-   `Main` is the entry point of your program which will read in the
    data, run the classifiers and print the results. Note that
    pre-processing has been done for you; feel free to examine the code
    for what exactly was done.

You should *not* modify any code in the `Main`. All the functions you
need to fill in reside in `Models`, marked by `TODO`s. You can see a
full description of them in the section below. 

### German Credit Dataset

You will be using the commonly-used German Credit dataset, which
includes 1000 total examples. The prediction task is to decide whether
someone's credit is good (1) or bad (0). A full list of attributes can
be found
[**here**](https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29);
note that this includes sensitive attributes like sex, age, and personal
status. The specific file we are using comes from [**Friedler et.al.,
2019**](https://github.com/algofairness/fairness-comparison). This data
is in the file `german_numerical-binsensitive.csv`.

### Data Format

The original feature values in this dataset are mixed---some
categorical, some numerical. We have written all the preprocessing code
for you, transforming numerical attributes into categories and encoding
all attributes as binary features. After preprocessing, there are a
total of 69 attributes which take on either 1 or 0. **`credit = 1`
corresponds to "good\" credit, and `credit = 0` corresponds to "bad\"
credit.**

## **The Assignment**

In `Models`, there are three functions you will implement. They are:

-   `NaiveBayes`:

    -   **train()** uses maximum likelihood estimation to learn the
        parameters (attribute distributions and priors distribution).
        Because all the features are binary values, you should use the
        Bernoulli distribution (as described in lecture) for the
        features. Remember to add Laplace smoothing as you calculate the
        distributions.

    -   **predict()** predicts the labels using the inputs of test data.
        You should return 1-D numpy array.

    -   **accuracy()** computes the percentage of the correctly
        predicted labels over a dataset.

Note that there is also a **print_fairness()** method implemented for
you in `NaiveBayes`. You should not change this method. Additionally,
you are not allowed to use any off-the-shelf packages that have already
implemented Naive Bayes, such as scikit-learn; we're asking you to
implement it yourself.

In [1]:
#[TODO] run this cell to make sure you are in the right environment. 
# We will deduct 2 points for each missing OK sign.
from __future__ import print_function
from packaging.version import parse as Version
from platform import python_version

OK = '\x1b[42m[ OK ]\x1b[0m'
FAIL = "\x1b[41m[FAIL]\x1b[0m"

try:
    import importlib
except ImportError:
    print(FAIL, "Python version 3.10 is required,"
                " but %s is installed." % sys.version)

def import_version(pkg, min_ver, fail_msg=""):
    mod = None
    try:
        mod = importlib.import_module(pkg)
        if pkg in {'PIL'}:
            ver = mod.VERSION
        else:
            ver = mod.__version__
        if Version(ver) == Version(min_ver):
            print(OK, "%s version %s is installed."
                  % (lib, min_ver))
        else:
            print(FAIL, "%s version %s is required, but %s installed."
                  % (lib, min_ver, ver))    
    except ImportError:
        print(FAIL, '%s not installed. %s' % (pkg, fail_msg))
    return mod


# first check the python version
pyversion = Version(python_version())

if pyversion >= Version("3.10.7"):
    print(OK, "Python version is %s" % pyversion)
elif pyversion < Version("3.10.7"):
    print(FAIL, "Python version 3.10.7 is required,"
                " but %s is installed." % pyversion)
else:
    print(FAIL, "Unknown Python version: %s" % pyversion)

    
print()
requirements = {'matplotlib': "3.7.2", 'numpy': "1.24.4",'sklearn': "1.3.0", 
                'pandas': "2.0.3", "pytest": "7.2.1"}

# now the dependencies
for lib, required_version in list(requirements.items()):
    import_version(lib, required_version)

[ OK ] Python version is 3.10.7

[ OK ] matplotlib version 3.6.0 is installed.
[ OK ] numpy version 1.23.3 is installed.
[ OK ] sklearn version 1.1.1 is installed.
[ OK ] pandas version 1.4.2 is installed.


## **Model**

In [2]:
import numpy as np
import pandas as pd

class NaiveBayes(object):
    """ Bernoulli Naive Bayes model
    
    @attrs:
        n_classes:    the number of classes
        attr_dist:    a 2D (n_classes x n_attributes) NumPy array of the attribute distributions
        label_priors: a 1D NumPy array of the priors distribution
    """

    def __init__(self, n_classes):
        """ Initializes a NaiveBayes model with n_classes. """
        self.n_classes = n_classes
        self.attr_dist = None
        self.label_priors = None

    def train(self, X_train, y_train):
        """ Trains the model, using maximum likelihood estimation.
        @params:
            X_train: a 2D (n_examples x n_attributes) numpy array
            y_train: a 1D (n_examples) numpy array
        @return:
            a tuple consisting of:
                1) a 2D numpy array of the attribute distributions
                2) a 1D numpy array of the priors distribution
        """

        # TODO
        n_examples = len(X_train)
        n_attributes = len(X_train[0])

        # Priors, 1st smoothing
        self.label_priors = np.zeros(2)
        self.label_priors[0] = (n_examples-sum(y_train)+1)/(n_examples+2) 
        self.label_priors[1] = (sum(y_train)+1)/(n_examples+2) 

        # Attributes, 2nd smoothing
        self.attr_dist = np.zeros((n_attributes, 2))
        class0 = X_train[y_train==0]
        class1 = X_train[y_train==1]
        for attr in range(n_attributes):
            self.attr_dist[attr,0] = (sum(class0[:,attr])+1)/(len(class0)+2)
            self.attr_dist[attr,1] = (sum(class1[:,attr])+1)/(len(class1)+2)
        return self.attr_dist.T, self.label_priors

    def predict(self, inputs):
        """ Outputs a predicted label for each input in inputs.
            Remember to convert to log space to avoid overflow/underflow
            errors!

        @params:
            inputs: a 2D NumPy array containing inputs
        @return:
            a 1D numpy array of predictions
        """

        # TODO
        predictions = np.zeros(len(inputs))
        for i in range(len(inputs)):
            input = inputs[i]
            prob_table = np.zeros(np.shape(self.attr_dist))
            prob_table[input==1] = self.attr_dist[input==1]
            prob_table[input==0] = 1-self.attr_dist[input==0]
            log_prob_table = np.log(prob_table)
            prob = np.exp(log_prob_table.sum(axis=0)) * self.label_priors
            predictions[i] = np.argmax(prob)
        return predictions

    def accuracy(self, X_test, y_test):
        """ Outputs the accuracy of the trained model on a given dataset (data).

        @params:
            X_test: a 2D numpy array of examples
            y_test: a 1D numpy array of labels
        @return:
            a float number indicating accuracy (between 0 and 1)
        """

        # TODO
        predictions = self.predict(X_test)
        accuracy = sum(predictions==y_test)/len(y_test)
        return accuracy

    def print_fairness(self, X_test, y_test, x_sens):
        """ 
        ***DO NOT CHANGE what we have implemented here.***
        
        Prints measures of the trained model's fairness on a given dataset (data).

        For all of these measures, x_sens == 1 corresponds to the "privileged"
        class, and x_sens == 0 corresponds to the "disadvantaged" class. Remember that
        y == 1 corresponds to "good" credit. 

        @params:
            X_test: a 2D numpy array of examples
            y_test: a 1D numpy array of labels
            x_sens: a numpy array of sensitive attribute values
        @return:

        """
        predictions = self.predict(X_test)

        # Disparate Impact (80% rule): A measure based on base rates: one of
        # two tests used in legal literature. All unprivileged classes are
        # grouped together as values of 0 and all privileged classes are given
        # the class 1. . Given data set D = (S,X,Y), with protected
        # attribute S (e.g., race, sex, religion, etc.), remaining attributes X,
        # and binary class to be predicted Y (e.g., “will hire”), we will say
        # that D has disparate impact if:
        # P[Y^ = 1 | S != 1] / P[Y^ = 1 | S = 1] <= (t = 0.8). 
        # Note that this 80% rule is based on US legal precedent; mathematically,
        # perfect "equality" would mean

        di = np.mean(predictions[np.where(x_sens==0)])/np.mean(predictions[np.where(x_sens==1)])
        print("Disparate impact: " + str(di))

        # Group-conditioned error rates! False positives/negatives conditioned on group
        
        pred_priv = predictions[np.where(x_sens==1)]
        pred_unpr = predictions[np.where(x_sens==0)]
        y_priv = y_test[np.where(x_sens==1)]
        y_unpr = y_test[np.where(x_sens==0)]

        # s-TPR (true positive rate) = P[Y^=1|Y=1,S=s]
        priv_tpr = np.sum(np.logical_and(pred_priv == 1, y_priv == 1))/np.sum(y_priv)
        unpr_tpr = np.sum(np.logical_and(pred_unpr == 1, y_unpr == 1))/np.sum(y_unpr)

        # s-TNR (true negative rate) = P[Y^=0|Y=0,S=s]
        priv_tnr = np.sum(np.logical_and(pred_priv == 0, y_priv == 0))/(len(y_priv) - np.sum(y_priv))
        unpr_tnr = np.sum(np.logical_and(pred_unpr == 0, y_unpr == 0))/(len(y_unpr) - np.sum(y_unpr))

        # s-FPR (false positive rate) = P[Y^=1|Y=0,S=s]
        priv_fpr = 1 - priv_tnr 
        unpr_fpr = 1 - unpr_tnr 

        # s-FNR (false negative rate) = P[Y^=0|Y=1,S=s]
        priv_fnr = 1 - priv_tpr 
        unpr_fnr = 1 - unpr_tpr

        print("FPR (priv, unpriv): " + str(priv_fpr) + ", " + str(unpr_fpr))
        print("FNR (priv, unpriv): " + str(priv_fnr) + ", " + str(unpr_fnr))
    
    
        # #### ADDITIONAL MEASURES IF YOU'RE CURIOUS #####

        # Calders and Verwer (CV) : Similar comparison as disparate impact, but
        # considers difference instead of ratio. Historically, this measure is
        # used in the UK to evalutate for gender discrimination. Uses a similar
        # binary grouping strategy. Requiring CV = 1 is also called demographic
        # parity.

        cv = 1 - (np.mean(predictions[np.where(x_sens==1)]) - np.mean(predictions[np.where(x_sens==0)]))

        # Group Conditioned Accuracy: s-Accuracy = P[Y^=y|Y=y,S=s]

        priv_accuracy = np.mean(predictions[np.where(x_sens==1)] == y_test[np.where(x_sens==1)])
        unpriv_accuracy = np.mean(predictions[np.where(x_sens==0)] == y_test[np.where(x_sens==0)])

        return predictions


## **Check Model**

In [22]:
import pytest
# Sets random seed for testing purposes
np.random.seed(0)

# Creates Test Models with 2 classes
test_model1 = NaiveBayes(2)
test_model2 = NaiveBayes(2)

# Creates Test Data
x = np.array([[0,0,1], [0,1,0], [1,0,1], [1,1,1], [0,0,1]])
y = np.array([0,0,1,1,0])
x_test = np.array([[1,0,0],[0,0,0],[1,1,1],[0,1,0], [1,1,0]])
y_test = np.array([0,0,1,0,1])

x2 = np.array([[0,0,1], [0,1,1], [1,1,1], [1,1,1], [0,0,0], [1,1,0]])
y2 = np.array([0,1,1,1,0,1])
x_test2 = np.array([[0,0,1], [0,1,1], [1,1,1], [1,0,0]])
y_test2 = np.array([0,1,1,0])

# Test Model Train
assert (test_model1.train(x,y)[0] ==  np.array([[.2, .4, .6],[.75, .5, .75]])).all()
assert test_model1.train(x,y)[1] == pytest.approx(np.array([0.571, 0.429]), 0.01)
assert (test_model2.train(x2, y2)[0] ==  pytest.approx(np.array([[.25, .25, .5],[.67, .83, .67]]), 0.01))
assert test_model2.train(x2,y2)[1] == pytest.approx(np.array([0.375, 0.625]), 0.01)

# Test Model Predict
assert (test_model1.predict(x_test) == np.array([1., 0., 1., 0., 1.])).all()
assert (test_model2.predict(x_test2) == np.array([0, 1, 1, 0])).all()

# Test Model Accuracy
assert test_model1.accuracy(x_test, y_test) == .8
assert test_model2.accuracy(x_test2, y_test2) == 1.0

## **Main**

In [4]:
def get_credit():
    """
    Gets and preprocesses German Credit data
    """
    data = pd.read_csv('./data/german_numerical-binsensitive.csv') # Reads file - may change

    # MONTH categorizing
    data['month'] = pd.cut(data['month'],3, labels=['month_1', 'month_2', 'month_3'], retbins=True)[0]
    # month bins: [ 3.932     , 26.66666667, 49.33333333, 72.        ]
    a = pd.get_dummies(data['month'])
    data = pd.concat([data, a], axis = 1)
    data = data.drop(['month'], axis=1)

    # CREDIT categorizing
    data['credit_amount'] = pd.cut(data['credit_amount'], 3, labels=['cred_amt_1', 'cred_amt_2', 'cred_amt_3'], retbins=True)[0]
    # credit bins: [  231.826,  6308.   , 12366.   , 18424.   ]
    a = pd.get_dummies(data['credit_amount'])
    data = pd.concat([data, a], axis = 1)
    data = data.drop(['credit_amount'], axis=1)

    for header in ['investment_as_income_percentage', 'residence_since', 'number_of_credits']:
        a = pd.get_dummies(data[header], prefix=header)
        data = pd.concat([data, a], axis = 1)
        data = data.drop([header], axis=1)

    # change from 1-2 classes to 0-1 classes
    data['people_liable_for'] = data['people_liable_for'] -1
    data['credit'] = -1*(data['credit']) + 2 # original encoding 1: good, 2: bad. we switch to 1: good, 0: bad

    # balance dataset
    data = data.reindex(np.random.permutation(data.index)) # shuffle
    pos = data.loc[data['credit'] == 1]
    neg = data.loc[data['credit'] == 0][:350]
    combined = pd.concat([pos, neg])

    y = data.iloc[:, data.columns == 'credit'].to_numpy()
    x = data.drop(['credit', 'sex', 'age', 'sex-age'], axis=1).to_numpy()

    # split into train and validation
    X_train, X_val, y_train, y_val = x[:350, :], x[351:526, :], y[:350, :].reshape([350,]), y[351:526, :].reshape([175,])

    # keep info about sex and age of validation rows for fairness portion
    x_sex = data.iloc[:, data.columns == 'sex'].to_numpy()[351:526].reshape([175,])
    x_age = data.iloc[:, data.columns == 'age'].to_numpy()[351:526].reshape([175,])
    x_sex_age = data.iloc[:, data.columns == 'sex-age'].to_numpy()[351:526].reshape([175,])

    return X_train, X_val, y_train, y_val, x_sex, x_age, x_sex_age


np.random.seed(0)
X_train, X_val, y_train, y_val, x_sex, x_age, x_sex_age = get_credit()
model = NaiveBayes(2)
model.train(X_train, y_train)

print("------------------------------------------------------------")
print("Train accuracy:")
print(model.accuracy(X_train, y_train))
print("------------------------------------------------------------")
print("Test accuracy:")
print(model.accuracy(X_val, y_val))
print("------------------------------------------------------------")

print("Fairness measures:")
model.print_fairness(X_val, y_val, x_sex_age)


------------------------------------------------------------
Train accuracy:
0.7742857142857142
------------------------------------------------------------
Test accuracy:
0.7257142857142858
------------------------------------------------------------
Fairness measures:
Disparate impact: 0.8294586797895808
FPR (priv, unpriv): 0.7083333333333333, 0.37037037037037035
FNR (priv, unpriv): 0.17500000000000004, 0.15909090909090906


array([1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1.,
       1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 0.])

## **Project Report**

### **Question 1**

Report the training and testing accuracy of the Naive Bayes
classifier. (A correct implementation should have testing accuracy
above $70\%$.)

**Solution:**

### **Question 2**

What strong assumption about the features/attributes of the data
does Naive Bayes make? Comment on this assumption in the context of
credit scores.

**Solution:** Every feature is independent. Probably doesn't hold
true in the context of credit scores (but hey still better than
mnist?)

### **Question 3**

This dataset was originally structured as follows:


| Month | Credit Amount | Number of credits | ... | Credit |
| ------| ------------- | ----------------- | --- | ------ |
|   6   |      1169     |         2         | ... |    1   |
|   48  |      5951     |         1         | ... |    2   |
|   12  |      2096     |         1         | ... |    1   | 
|   9   |      2134     |         3         | ... |    1   |

For each of the above attributes, describe what transformations to
the original dataset would need to occur for it to be usable in a
Bernoulli Naive Bayes model. *(hint: every attribute must take on
the value of 0 or 1)* 

**Solution:**

Discretize "month" and "credit amount"

Binarize "month," "credit amount," "number of credits"

Switch "credit" encoding to 0-1

### **Question 4**

A different way to think about fairness is based on the errors the
model makes. We define the false positive rate (FPR) as
$P(\hat Y = 1 | Y = 0)$, and the false negative rate (FNR) as
$P(\hat Y = 0 | Y = 1)$. Suppose we calculate FPR and FNR for each
group. In words, what does the false positive rate and false
negative rate represent in the context of credit ratings? What are
the implications if one group's FPR is much higher than the other's?
What are the implications if one group's FNR is much higher than the
other's? 

**Solution:**

FPR: f the time someone was given \"good credit\" when
they actually had \"bad credit\"

FNR: of the time someone was given \"bad credit\" when they actually
had \"good credit\"

FPR disparity: one group gets more access to credit/loans/etc than
they should, unfairly rewarding those who are \"undeserving\" in the
group with higher FPR

FNR disparity: one group gets less access to credit/loans/etc than
they should, unfairly punishing those who are \"deserving\" in the
group with higher FNR